In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
     
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import json
import os
import re

import gensim 
from gensim.models import Word2Vec
import gensim.downloader

import pickle 

import nltk
from nltk.tokenize import word_tokenize

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

import pickle

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
import flair
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings

import sys

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
tqdm.pandas()

# Opening Files: 

In [4]:
# Open Pickle without lemmatisation: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, Y_train, Y_test, _, _) = pickle.load(handle)

In [5]:
# Open Pickle lemmatised: 

#with open('merged_data_lemma.pickle', 'rb') as handle:
#    (X_train_text, X_test_text, Y_train, Y_test, _, _) = pickle.load(handle)

In [8]:
# Stemming: 

#stemmer = SnowballStemmer(language='english')
#df['stemmer'] = df['text'].apply(lambda x: " ".join([stemmer.stem(token) for token in x]))


# Feature Extraction:

In [5]:
sys.path.append('../src')

In [6]:
# call method
from methods import feature_extraction, evaluation

In [7]:
pd.DataFrame([1,2,3,4]).reset_index().apply(lambda row: print(row.index),axis=1)

Index(['index', 0], dtype='object')
Index(['index', 0], dtype='object')
Index(['index', 0], dtype='object')
Index(['index', 0], dtype='object')


0    None
1    None
2    None
3    None
dtype: object

## Count Vectorizer:

In [18]:
#X_train, X_test = feature_extraction('CountVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## TF-IDF:

In [43]:
#X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## word2vec Google news:

In [ ]:
w2v_google = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
X_train, X_test = feature_extraction('embedding', X_train_text, X_test_text, embedding_type = w2v_google, weighted=True)

## Glove:

In [ ]:
#glv = gensim.downloader.load('glove-wiki-gigaword-100')

In [ ]:
# X_train, X_test = feature_extraction('embedding', X_train_text, X_test_text, embedding_type = glv)

## Trained word2vec:

In [56]:
#w2v = Word2Vec.load("word2vec.model").wv

In [57]:
#X_train, X_test = feature_extraction("embedding", X_train_text, X_test_text, embedding_type = w2v, weighted=False)

2072it [01:56, 17.77it/s]
441it [00:14, 30.16it/s]


# Classifiers:

## Naive Bayes:

In [ ]:
naive_bayes_classifier = OneVsRestClassifier(MultinomialNB())
naive_bayes_classifier.fit(X_train, Y_train)

In [ ]:
y_pred_proba = pd.DataFrame(naive_bayes_classifier.predict_proba(X_test), columns = Y_test.columns)

In [ ]:
y_pred = (y_pred_proba > 0.005).astype(int) # if increase threshold, recall decreases and precision (could) increase

In [ ]:
evaluation(y_pred, Y_test)

## SVC:

In [ ]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


In [ ]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [ ]:
evaluation(Y_pred, Y_test)

## Multi-label KNN: 

In [14]:
knn = MLkNN(k = 3)

In [ ]:
# only works old version sklearn 

In [15]:
# train
knn.fit(X_train.values, Y_train.values)

# predict
predictions = knn.predict(X_test.values)

TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
evaluation(predictions, Y_test)

## Logistic Regression:

In [ ]:
# reduce dimension using pca: 

pca = PCA(n_components=75)
pca.fit(X_train)
pca_result = pca.transform(X_train)
x_test_result = pca.transform(X_test )

In [ ]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0, multi_class='multinomial', solver='lbfgs', max_iter = 1000)).fit(X_train, Y_train)

predictions = log_reg.predict(X_test)


In [ ]:
evaluation(predictions, Y_test)

## DT AdaBoost: 

In [ ]:
dt_adaboost = OneVsRestClassifier(AdaBoostClassifier(n_estimators=100, random_state=0)).fit(X_train, Y_train)
predictions_ada = dt_adaboost.predict(X_test)

In [ ]:
evaluation(predictions_ada, Y_test)

## Random Forest:

In [31]:
predictions_rf = OneVsRestClassifier(RandomForestClassifier(max_depth=2, random_state=0)).fit(X_train, Y_train)

In [32]:
evaluation(predictions_rf, Y_test)

TypeError: Expected sequence or array-like, got <class 'sklearn.multiclass.OneVsRestClassifier'>

# Classifer Chain: 

In [52]:
def chain_model(model):
    model_chain = ClassifierChain(model, order='random', random_state=0)
    return model_chain

In [ ]:
#chain = chain_model(naive_bayes_classifier) # change model appropriately

In [53]:
chain = chain_model(log_reg) # change model appropriately

In [54]:
chainModel = chain.fit(X_train.values, Y_train.values)
predictions = chainModel.predict(X_test.values)

In [55]:
evaluation(predictions, Y_test)

,metric,result
0,macro precision,0.704028
1,micro precision,0.700301
2,macro recall,0.303006
3,micro recall,0.380213
4,macro fscore,0.498876
5,micro fscore,0.599381


# Neural Networks:

## Multi Layer Perceptron: 

In [17]:
mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, Y_train)

In [18]:
predictions_mlp = mlp.predict(X_test)

In [19]:
evaluation(predictions_mlp, Y_test)

,metric,result
0,macro precision,0.357294
1,micro precision,0.433931
2,macro recall,0.351384
3,micro recall,0.434996
4,macro fscore,0.345899
5,micro fscore,0.434144


## Loading data from flair:

In [10]:
with open('fasttext_format_test.txt', 'w') as file:
    for i in range(len(Y_test)):
        file.write(' '.join(['__label__'+col for col in Y_test.columns if Y_test.iloc[i][col] == 1]) + ' ' + X_test_text.iloc[i] + '\n')

In [11]:

flair.device = 'cpu'
# this is the folder in which train, test and dev files reside
data_folder = '.'

# load corpus containing training, test and dev data
corpus = ClassificationCorpus(data_folder,
                                      test_file='fasttext_format_test.txt',
                                      dev_file='fasttext_format_test.txt',
                                      train_file='fasttext_format_train.txt',
                                      label_type='tactic',
                                      )

2022-08-04 12:19:25,624 Reading data from .
2022-08-04 12:19:25,625 Train: fasttext_format_train.txt
2022-08-04 12:19:25,625 Dev: fasttext_format_test.txt
2022-08-04 12:19:25,626 Test: fasttext_format_test.txt
2022-08-04 12:19:26,403 Initialized corpus . (label type name is 'tactic')


In [ ]:
# 2. what label do we want to predict?
label_type = 'tactic'

# 3. create the label dictionary
label_dict = corpus.make_label_dictionary(label_type=label_type)

2022-08-04 12:19:26,407 Computing label dictionary. Progress:


2135it [02:27, 18.93it/s]

## Transformers:

In [ ]:
# initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('binay1999/text_classification_cybertexts', fine_tune=True)

# create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type, multi_label=True)

# initialize trainer
trainer = ModelTrainer(classifier, corpus)

# run training with fine-tuning
trainer.fine_tune('test_model',
                  learning_rate=5.0e-5,
                  mini_batch_size=4,
                  max_epochs=10,
                  )

2022-08-04 09:43:20,996 ----------------------------------------------------------------------------------------------------
2022-08-04 09:43:20,997 Model: "TextClassifier(
  (decoder): Linear(in_features=768, out_features=13, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_feat

2022-08-04 10:09:21,578 epoch 1 - iter 265/538 - loss 0.14704164 - samples/sec: 0.79 - lr: 0.000025
2022-08-04 10:13:42,135 epoch 1 - iter 318/538 - loss 0.13728537 - samples/sec: 0.84 - lr: 0.000030
2022-08-04 10:18:13,999 epoch 1 - iter 371/538 - loss 0.12862855 - samples/sec: 0.80 - lr: 0.000034
2022-08-04 10:22:56,452 epoch 1 - iter 424/538 - loss 0.12747004 - samples/sec: 0.79 - lr: 0.000039
2022-08-04 10:28:20,600 epoch 1 - iter 477/538 - loss 0.12901609 - samples/sec: 0.76 - lr: 0.000044
2022-08-04 10:33:25,817 epoch 1 - iter 530/538 - loss 0.12956674 - samples/sec: 0.78 - lr: 0.000049
2022-08-04 10:34:06,002 ----------------------------------------------------------------------------------------------------
2022-08-04 10:34:06,003 EPOCH 1 done: loss 0.1295 - lr 0.000049


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [03:08<00:00,  1.70s/it]

2022-08-04 10:37:14,246 Evaluating as a multi-label problem: True
2022-08-04 10:37:14,279 DEV : loss 0.17119154334068298 - f1-score (micro avg)  0.4246


2022-08-04 10:37:35,765 BAD EPOCHS (no improvement): 4
2022-08-04 10:37:35,767 ----------------------------------------------------------------------------------------------------
2022-08-04 10:42:24,557 epoch 2 - iter 53/538 - loss 0.13295517 - samples/sec: 0.80 - lr: 0.000049
2022-08-04 10:46:59,710 epoch 2 - iter 106/538 - loss 0.13127522 - samples/sec: 0.81 - lr: 0.000049
2022-08-04 10:51:41,020 epoch 2 - iter 159/538 - loss 0.12856716 - samples/sec: 0.80 - lr: 0.000048
2022-08-04 10:56:31,290 epoch 2 - iter 212/538 - loss 0.12789267 - samples/sec: 0.79 - lr: 0.000048
2022-08-04 11:01:16,526 epoch 2 - iter 265/538 - loss 0.12768929 - samples/sec: 0.80 - lr: 0.000047
2022-08-04 11:06:15,114 epoch 2 - iter 318/538 - loss 0.12716793 - samples/sec: 0.77 - lr: 0.000047
2022-08-04 11:11:07,052 epoch 2 - iter 371/538 - loss 0.12679577 - samples/sec: 0.78 - lr: 0.000046
2022-08-04 11:15:47,842 epoch 2 - iter 424/538 - loss 0.12652102 - samples/sec: 0.80 - lr: 0.000046
2022-08-04 11:20:24,9

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [03:10<00:00,  1.72s/it]

2022-08-04 11:28:55,163 Evaluating as a multi-label problem: True
2022-08-04 11:28:55,194 DEV : loss 0.12129916250705719 - f1-score (micro avg)  0.2634


2022-08-04 11:29:17,562 BAD EPOCHS (no improvement): 4
2022-08-04 11:29:17,564 ----------------------------------------------------------------------------------------------------
2022-08-04 11:33:48,121 epoch 3 - iter 53/538 - loss 0.11967454 - samples/sec: 0.82 - lr: 0.000044
2022-08-04 11:38:36,493 epoch 3 - iter 106/538 - loss 0.12186724 - samples/sec: 0.79 - lr: 0.000043
2022-08-04 11:43:18,591 epoch 3 - iter 159/538 - loss 0.12146273 - samples/sec: 0.81 - lr: 0.000043
2022-08-04 11:47:42,910 epoch 3 - iter 212/538 - loss 0.12198870 - samples/sec: 0.84 - lr: 0.000042
2022-08-04 11:52:28,279 epoch 3 - iter 265/538 - loss 0.12293185 - samples/sec: 0.81 - lr: 0.000042
2022-08-04 11:57:25,063 epoch 3 - iter 318/538 - loss 0.12262863 - samples/sec: 0.78 - lr: 0.000041


## LSTM (with word2vec):

In [13]:
embedding = WordEmbeddings('en')

document_embeddings = DocumentRNNEmbeddings([embedding])

In [ ]:
# create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type, multi_label=True)

# initialize trainer
trainer = ModelTrainer(classifier, corpus)

# run training with fine-tuning
trainer.fine_tune('test_model_word2vec',
                  learning_rate=5.0e-5,
                  mini_batch_size=4,
                  max_epochs=10,
                  )

2022-08-04 12:22:12,250 ----------------------------------------------------------------------------------------------------
2022-08-04 12:22:12,251 Model: "TextClassifier(
  (decoder): Linear(in_features=128, out_features=13, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (locked_dropout): LockedDropout(p=0.0)
  (word_dropout): WordDropout(p=0.0)
  (loss_function): BCEWithLogitsLoss()
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'en'
        (embedding): Embedding(1000001, 300)
      )
    )
    (word_reprojection_map): Linear(in_features=300, out_features=300, bias=True)
    (rnn): GRU(300, 128, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (weights): None
  (weight_tensor) None
)"
2022-08-04 12:22:12,252 ----------------------------------------------------------------------------------------------------
2022-08-04 12:22:12,253 Corpus: "Corpus: 2152 train + 441 de